In [1]:
# LangChain Tools
# 외부 기능이나  api 
# 주식, 날시, 웹 검색 ....
# llm 훈련데이터 이후의 정보는 모른다 - 웹 검색
# 외부 API 호출 Tool 구현

In [ ]:
%pip install langchain-core langchain-openai openai python-dotenv requests

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()
print(os.getenv('OPENAI_API_KEY')[:5])
print(os.getenv('OWM_API_KEY')[:5])

sk-pr
ff3b4


In [11]:
import requests
api_key = os.getenv('OWM_API_KEY')
url = 'https://api.openweathermap.org/data/2.5/weather'
city = 'Seoul'
params = {
    'q':city,
    'appid':api_key,
    'units':'metric',  # 섭씨온도
    'lang' : 'kr'
}
response = requests.get(url,params=params)
data = response.json()
if data.get('cod') != 200:
    print(f'{city}의 날씨 정보를 찾을 수 없습니다.')
# 필요한 정보를 파싱
temp = data['main']['temp']
desc = data['weather'][0]['description']
city_name = data['name']
print(f'{city_name}의 현재 기온은 {temp}c , 날씨는 {desc} 입니다.' )

Seoul의 현재 기온은 32.2c , 날씨는 튼구름 입니다.
